## Description:

Gets and stores data for nodes (representing street segments)

In [ ]:
import osmnx as ox
import folium
from geopy.distance import geodesic
from shapely.geometry import Polygon


def filter_edges_by_grid(edges, grid_square):
    print("filtering edges")
    west, south, east, north = grid_square
    grid_polygon = Polygon([(west, south), (west, north), (east, north), (east, south)])
    
    # Use the GeoDataFrame's spatial methods to check if edge geometries intersect with the grid square
    filtered_edges = edges[edges.geometry.apply(lambda geom: geom.intersects(grid_polygon))]
    return filtered_edges


# Function to interpolate points every 10 feet
def interpolate_points(start, end, step_ft):
    points = [start]
    total_distance = geodesic(start, end).feet
    step_ratio = step_ft / total_distance
    while step_ratio < 1.0:
        interp_lat = start[0] + (end[0] - start[0]) * step_ratio
        interp_lon = start[1] + (end[1] - start[1]) * step_ratio
        points.append((interp_lat, interp_lon))
        step_ratio += step_ft / total_distance
    points.append(end)  # Ensure the end point is included
    return points

def create_map_for_edges(edges):
    global grid_counter

    FEET_SPACING
    grid_counter += 1
    print("Creating map for Edges")
    point_counter = 0

    # Initialize a Folium map
    m = folium.Map(location=center_point, zoom_start=16)
    
    # Add edges to the map
    for _, row in edges.iterrows():
        coords = [(y, x) for x, y in row['geometry'].coords]
        
        # Interpolate points between each pair of coordinates
        for i in range(len(coords) - 1):
            start = coords[i]
            end = coords[i + 1]
            interpolated_points = interpolate_points(start, end, step_ft=FEET_SPACING)
            
            # Plot each interpolated point
            for point in interpolated_points:
                point_counter += 1
                # folium.Marker(location=point, popup=f"({location[0]}, {location[1]})").add_to(mymap)
                folium.CircleMarker(
                    location=point,
                    radius=2,
                    color='red',
                    fill=True,
                    fill_opacity=0.8,
                    popup=f"({point[0]}, {point[1]})"
                ).add_to(m)
    
    # Save the map
    m.save(f"data/dc_nodes/dc_{grid_counter}.html")
    print(f"Map created with {point_counter} points.")


def main():

    print("Starting")

    graph = ox.graph_from_place("Washington, DC, USA", network_type='drive')

    print("getting nodes and edges")
    nodes, edges = ox.graph_to_gdfs(graph)

    # prep for batching (since these is way to large of a dataset)
    bbox = edges.total_bounds  # [west, south, east, north]
    print("got bounds")


    # Divide the bounding box into 20 grids (4x5)
    num_rows = 6
    num_cols = 7
    grid_width = (bbox[2] - bbox[0]) / num_cols  # (east - west) / cols
    grid_height = (bbox[3] - bbox[1]) / num_rows  # (north - south) / rows

    grid_squares = []
    print("creating square")
    for row in range(num_rows):
        for col in range(num_cols):
            west = bbox[0] + col * grid_width
            east = west + grid_width
            south = bbox[1] + row * grid_height
            north = south + grid_height
            grid_squares.append((west, south, east, north))

    print(f"grid squares: {len(grid_squares)}")
    for idx, grid_square in enumerate(grid_squares):
        filtered_edges = filter_edges_by_grid(edges, grid_square)
        create_map_for_edges(filtered_edges)

grid_counter = 0
FEET_SPACING = 100
center_point = (38.90251234580012, -77.04350506490759)


main()


In [ ]:
import osmnx as ox
point = (38.90373,-77.0488552)
# graph = ox.graph_from_place("Washington, DC, USA", network_type='drive')
graph = ox.graph_from_point(point,1000, network_type='drive')

nodes, edges = ox.graph_to_gdfs(graph)





In [ ]:
import folium

# Create a folium map centered around the average latitude and longitude of the nodes
center_lat = nodes.geometry.y.mean()
center_lon = nodes.geometry.x.mean()
dc_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add a marker for each node
for node_id, node in nodes.iterrows():
    print(node)
    lat = node.geometry.y
    lon = node.geometry.x
    # folium.Marker(location=[lat, lon], popup=f"Node ID: {node_id}").add_to(dc_map)
    folium.CircleMarker(
                    location=[lat, lon],
                    radius=2,
                    color='red',
                    fill=True,
                    fill_opacity=0.8,
                    popup=f"({point[0]}, {point[1]})"
                ).add_to(dc_map)


# Add edges to the map with different colors for one-way and two-way edges
for _, edge in edges.iterrows():
    # Extract edge geometry
    if edge.geometry.geom_type == 'LineString':             
        coords = [(lat, lon) for lon, lat in edge.geometry.coords]
        color = 'red' if edge.get('oneway', False) else 'blue'
        folium.PolyLine(coords, color=color, weight=2.5, opacity=0.8).add_to(dc_map)
    elif edge.geometry.geom_type == 'MultiLineString':
        for line in edge.geometry:
            coords = [(lat, lon) for lon, lat in line.coords]
            color = 'red' if edge.get('oneway', False) else 'blue'
            folium.PolyLine(coords, color=color, weight=2.5, opacity=0.8).add_to(dc_map)

# Save or display the map
dc_map.save("data/test_washington_dc_nodes_map.html")
print("Saved")

# print(nodes)
# print(edges)

In [144]:
from folium.plugins import PolyLineTextPath
from folium import Icon
import math

def draw_segments_on_map(segments):
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    for segment_id, segment in segments.items():
        lat = segment['lat']
        long = segment['long']
        folium.CircleMarker(
            location=[lat, long],
            radius=2,
            color='red',
            fill=True,
            fill_opacity=0.8,
            popup=f" {segment_id}: => \nLinks:({segment['segment_links']}\nHeadings:({segment['headings']})"
        ).add_to(m)

        # Draw lines to each segment link
        for linked_segment_id in segment['segment_links']:
            linked_segment = segments.get(linked_segment_id)
            if linked_segment:
                linked_lat = linked_segment['lat']
                linked_long = linked_segment['long']
                
                # Create a PolyLine (line) between the current segment and the linked segment
                folium.PolyLine(
                    locations=[(lat, long), (linked_lat, linked_long)],
                    color='blue',
                    weight=2,
                    opacity=0.5
                ).add_to(m)

        # Draw lines for each heading
        for heading in segment['headings']:
            # Calculate the endpoint of the line for the heading
            distance = 0.0005  # Adjust distance for visual clarity on the map
            adjusted_heading = heading  # Adjust for clockwise from North to counterclockwise from East
            heading_rad = math.radians(adjusted_heading)

            # Calculate the end coordinates based on the heading
            end_lat = lat + distance * math.cos(heading_rad)  # Latitude change uses cos
            end_long = long + distance * math.sin(heading_rad)  # Longitude change uses sin

            # Draw a line indicating the direction of the heading
            folium.PolyLine(
                locations=[(lat, long), (end_lat, end_long)],
                color='green',
                weight=2,
                opacity=0.7
            ).add_to(m)

    m.save("data/dc_segments_map.html")

In [145]:
import math
import pandas as pd
from geopy.distance import geodesic
from shapely.geometry import LineString


def write_as_csv(filepath, dict):
    df = pd.DataFrame.from_dict(dict, orient='index')
    df.to_csv(filepath)

def segment_key(lat, long):
    return f"{lat}_{long}"


def calculate_heading(lat1, lon1, lat2, lon2):
    """
    Calculate the heading (bearing) from one point to another.
    Returns the heading in degrees.
    """
    d_lon = math.radians(lon2 - lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)

    x = math.sin(d_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))
    initial_heading = math.atan2(x, y)
    # Convert to degrees and normalize to 0-360
    heading = (math.degrees(initial_heading) + 360) % 360
    return heading

def add_all_nodes_to_segments(nodes, segments):
    # loop through all nodes and add them as segments
    for node_id, node in nodes.iterrows():
        lat = node.geometry.y
        long = node.geometry.x

        # print(node_id)
        # print(f"Node:{segment_key(lat,long)}")
        segment_headings = []
        segment_links = []

        successors = list(graph.successors(node_id))
        # print(successors)

        for successor_id in successors:
            successor_node = nodes.loc[successor_id]  
            successor_lat = successor_node.geometry.y
            successor_long = successor_node.geometry.x
            # print(f"  Successor Node ID: {successor_id}, Latitude: {successor_lat}, Longitude: {successor_long}")
            
            # # add segment link
            segment_links.append(segment_key(successor_lat, successor_long))

            # # calculate segment heading
            heading = calculate_heading(lat, long, successor_lat, successor_long)
            segment_headings.append(heading)


        # TODO debug
        segment_headings = []
        segment_links = []

        segments[segment_key(lat,long)] = {
            "lat": lat,
            "long": long, 
            "headings": segment_headings, 
            "segment_links": segment_links}

def flip_linestring_coords(line):
    # Extract the coordinates
    original_coords = list(line.coords)
    # Flip each coordinate pair
    flipped_coords = [(lat, lon) for lon, lat in original_coords]
    # Create a new LineString with flipped coordinates
    return LineString(flipped_coords)

def add_segment_to_segments(segments, lat, long, headings, segment_links):
    segments[segment_key(lat,long)] = {
        "lat": lat,
        "long": long, 
        "headings": headings, 
        "segment_links": segment_links}


def get_equally_spaced_points_from_edge(edge, num_points):
    # Extract the LineString geometry from the edge data
    geometry = edge[2].get('geometry')
    if not geometry:
        # No geometry data
        return []
    
    line = geometry  # The LineString
    total_length = line.length

    # Calculate distances at which to extract points
    distances = [i * total_length / (num_points - 1) for i in range(num_points)]
    
    # Extract points along the LineString
    spaced_points = [(line.interpolate(dist).y, line.interpolate(dist).x) for dist in distances]
    
    # Calculate headings between consecutive points
    points_with_headings = []
    for i in range(len(spaced_points) - 1):
        lat1, lon1 = spaced_points[i]
        lat2, lon2 = spaced_points[i + 1]
        heading = calculate_heading(lat1, lon1, lat2, lon2)
        points_with_headings.append((lat1, lon1, heading))
    
    # Add the last point without a heading (as there is no next point)
    points_with_headings.append((spaced_points[-1][0], spaced_points[-1][1], None))
    
    return points_with_headings


def get_total_feet_from_edge(edge):
    geometry = edge[2].get('geometry')
    if not geometry:
        # no geometry data... no locations
        return 0

    line = geometry
    length_in_feet = 0
    flipped_line = flip_linestring_coords(line)
    coords = list(flipped_line.coords)

    # Calculate the geodesic length segment by segment
    for i in range(len(coords) - 1):
        start = coords[i]
        end = coords[i + 1]
        segment_length = geodesic(start, end).feet
        print(f"Segment {i}: {start} -> {end}, Length in feet: {segment_length}")
        length_in_feet += segment_length
    
    return length_in_feet



def reverse_heading(heading):
    return (heading + 180) % 360


# attempts to add segments from an edge
def create_segments_from_edge(edge, segments, closed_edges):
    


    print(f"    u: {edge[0]}, v: {edge[1]}, data: {edge[2]}")
    if edge not in closed_edges:
        # we have not processed this edge yet...
        u = edge[0]
        v = edge[1]
        data = edge[2]

        oneway = bool(data['oneway'])

        # first get the distance between the two nodes
        u_node = nodes.loc[u]
        v_node = nodes.loc[v]
        u_lat = u_node.geometry.y
        u_long = u_node.geometry.x
        v_lat = v_node.geometry.y
        v_long = v_node.geometry.x

        u_key = segment_key(u_lat, u_long)
        v_key = segment_key(v_lat, v_long)
        
        u_heading = calculate_heading(u_lat, u_long, v_lat, v_long)
        # for two way streets we need to add the reverse heading
        v_heading = calculate_heading(v_lat, v_long, u_lat, u_long)

        #########
        FEET_SPACING = 200
        # how many segments do we need to add?
        total_feet = get_total_feet_from_edge(edge)

        print(f"TOTAL FEET: {total_feet}")

        num_segments_to_add = int(total_feet // FEET_SPACING) + 2 # add 2 to account for the start and end (we just remove them later)
        print(f"NUM SEGMENTS TO ADD: {num_segments_to_add}")
        print(get_equally_spaced_points_from_edge(edge, num_segments_to_add))
        segment_locations_headings = get_equally_spaced_points_from_edge(edge, num_segments_to_add)

        # remove the first and last segment location
        segment_locations_headings = segment_locations_headings[1:-1]

        # no locations to add? then make sure we link u to v (and potentially v to u)
        if len(segment_locations_headings) == 0:
            print("    NO SEGMENTS TO ADD")
            # cant add any intermediate segments, so we must connect the u and v nodes
            # no matter what, we need to link u to v
            print(f"    Adding Segment Link & header to: {u_key}:  links={v_key}, heading={u_heading}")
            segments[u_key]['segment_links'].append(v_key)
            segments[u_key]['headings'].append(u_heading)
            if not oneway:
                # then v can also point to u
                segments[v_key]['segment_links'].append(u_key)
                segments[v_key]['headings'].append(v_heading)
                
            # no segments to add, so we can end early
            return

        
        print(f"    Segment Locations: {segment_locations_headings}")
        for i, (lat, long, heading) in enumerate(segment_locations_headings):
            location = (lat, long)
            new_segment_lat = location[0]
            new_segment_long = location[1]
            segment_key_str = segment_key(new_segment_lat, new_segment_long)
            print(f"    Segment {i}: {location}")
            print(f"    {bool(data['oneway'])}")

            # default
            previous_segment = u_key
            next_segment = v_key

            segment_heading = u_heading
            reverse_segment_heading = v_heading

            if i != 0:
                # previous location segment...
                previous_segment = segment_key(segment_locations_headings[i - 1][0], segment_locations_headings[i - 1][1])
                segment_heading = heading
                reverse_segment_heading = reverse_heading(heading)

            else: 
                # we are the first segment, so we need to tell node 'u' to link to us, and point to us
                segments[u_key]['segment_links'].append(segment_key_str)
                segments[u_key]['headings'].append(heading)

            if i != len(segment_locations_headings) - 1:
                # we are not the last segment, 
                # next location segment...
                next_segment = segment_key(segment_locations_headings[i + 1][0], segment_locations_headings[i + 1][1])
                segment_heading = heading
                reverse_segment_heading = reverse_heading(heading)

            segment_links = []
            segment_headings = []

            
            

            # since we are are going in the outbound direction of a node (u -> v) we can assume
            # that the next segment in segment_locations is segment we need to link to (no matter what)
            # if we are not oneway (two way) then we need additionally need to link to the previous segment
            
            segment_links.append(next_segment)
            segment_headings.append(segment_heading)
            if not oneway:
                print("    NOT ONEWAY")
                print(f"    adding Previous Segment: {previous_segment}")
                segment_links.append(previous_segment)
                segment_headings.append(reverse_segment_heading)

            # add segment to segments
           
            print(f"    Creating Segment: {segment_key_str}, links: {segment_links}, headings: {segment_headings}")
            if segment_key_str not in segments:
                segments[segment_key_str] = {
                    "lat": new_segment_lat,
                    "long": new_segment_long, 
                    "headings": segment_headings, 
                    "segment_links": segment_links}
                

        # add edge to closed edges
        closed_edges.append(edge)



def add_edges_between_nodes_to_segments(nodes, segments):
    closed_edges = [] # list of edges that have already been processed

    # loop through all nodes and add segements between them
    for node_id, node in nodes.iterrows():
        print(f"For Node ID: {node_id}")
        lat = node.geometry.y
        long = node.geometry.x

        print(node_id)
        print(f"Node:{segment_key(lat,long)}")
        

        
        # print(successors)

        succesor_edges = graph.edges(node_id, data=True)
        # print(f"    {edges}")
        for edge in succesor_edges:
            create_segments_from_edge(edge, segments, closed_edges)
            
 
        # loop through all succesor edges and try to add segments within them





def grab_store_all_segments(nodes, edges):
    segments = {} #key = lat_long, value = lat, long, headings[], segment_links[]

    add_all_nodes_to_segments(nodes, segments)
    add_edges_between_nodes_to_segments(nodes, segments)

    
    print("PRINTING SEGMENTS:")
    print(segments)
    write_as_csv("data/dc_segments.csv", segments)
    draw_segments_on_map(segments)


grab_store_all_segments(nodes, edges)


For Node ID: 49716899
49716899
Node:38.9096436_-77.0477199
    u: 49716899, v: 739515930, data: {'osmid': [6051300, 6051301], 'highway': 'residential', 'name': 'Twining Court Northwest', 'oneway': False, 'reversed': [False, True], 'length': np.float64(82.75802668032613), 'geometry': <LINESTRING (-77.048 38.91, -77.048 38.91, -77.048 38.909, -77.047 38.909, -...>}
Segment 0: (38.9096436, -77.0477199) -> (38.9095596, -77.0477209), Length in feet: 30.595665812897288
Segment 1: (38.9095596, -77.0477209) -> (38.9092654, -77.0477246), Length in feet: 107.15821152124859
Segment 2: (38.9092654, -77.0477246) -> (38.9092675, -77.0473497), Length in feet: 106.68802964969518
Segment 3: (38.9092675, -77.0473497) -> (38.9092216, -77.0472746), Length in feet: 27.13314585835217
TOTAL FEET: 271.5750528421932
NUM SEGMENTS TO ADD: 3
[(38.9096436, -77.0477199, 175.5682015442692), (38.909265637199056, -77.0476822543218, 97.90346482327237), (38.9092216, -77.0472746, None)]
    Segment Locations: [(38.909265

[segmentid] = lat, long, headings[], segment_links[], 

segment_links -> which other segment ids can you get to from this one.
for example, on a one way street, you can only get to the next one, but on a two way, you can get to the previous and next one...

Note, some segment links may not have a respective google maps tile (and thats okay), we still want to keep track of links in case we want to calculate pathing

make each intersection its own segment